In [1]:
import pandas as pd
import ast
import requests
import time
import os
from collections import Counter
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

In [2]:
# =============================================================================
# LOAD DATA
# =============================================================================

# We load the CSV from Step 5 because it has the 'closing_method' column
path_to_file = 'output_files/prs_reviews_with_closure.csv'

if not os.path.exists(path_to_file):
    print(f"❌ Error: File not found: {path_to_file}")
    print("Please ensure you ran Step 5 successfully.")
else:
    # Read CSV (Dates are not critical here, but good practice)
    reviews = pd.read_csv(path_to_file)
    print(f"✅ Data loaded: {len(reviews)} rows.")

    # Select only necessary columns to save memory
    cols_to_keep = [
        'id', 'number', 'agent', 'repo_url', 'html_url', 
        'closing_method', 'reviews_data', 'review_counts_map', 
        'has_human_closing_user'
    ]
    # Check if columns exist
    if all(col in reviews.columns for col in cols_to_keep):
        reduced_reviews = reviews[cols_to_keep].copy()
        print("Columns filtered successfully.")
    else:
        print(f"⚠️ Warning: Some columns are missing. Available: {reviews.columns.tolist()}")
        reduced_reviews = reviews.copy()

✅ Data loaded: 2230 rows.
Columns filtered successfully.


In [3]:
# =============================================================================
# FILTERING
# =============================================================================

# 1. Filter PRs that have reviews (string length > 2 means it's not "[]")
prs_with_review = reduced_reviews[reduced_reviews['reviews_data'].str.len() > 2].copy()

# 2. Filter PRs that were REJECTED (Closed without merge)
rejected_prs = prs_with_review[prs_with_review['closing_method'] == 'Closed'].copy()

# 3. Filter PRs closed by a HUMAN
rejected_human_closed = rejected_prs[rejected_prs['has_human_closing_user'] == True].copy()

# 4. Filter PRs created by AI AGENTS (Target Population)
agentic_rejected_human_closed = rejected_human_closed[rejected_human_closed['agent'] != 'Human'].copy()

print(f"Total PRs found for analysis: {len(agentic_rejected_human_closed)}")
agentic_rejected_human_closed.head(2)

Total PRs found for analysis: 365


,id,number,agent,repo_url,html_url,closing_method,reviews_data,review_counts_map,has_human_closing_user
1,3164503419,40,Claude_Code,https://api.github.com/repos/amantus-ai/vibetu...,https://github.com/amantus-ai/vibetunnel/pull/40,Closed,"[{'user': 'coderabbitai[bot]', 'state': 'COMME...",{'coderabbitai[bot]': 2},True
12,3138264959,1831,Claude_Code,https://api.github.com/repos/evmts/tevm-monorepo,https://github.com/evmts/tevm-monorepo/pull/1831,Closed,"[{'user': 'coderabbitai[bot]', 'state': 'COMME...",{'coderabbitai[bot]': 1},True


In [4]:
# Create URLs for fetching comments
agentic_rejected_human_closed['review_comment_url'] = (
    agentic_rejected_human_closed['repo_url'] 
    + '/pulls/' 
    + agentic_rejected_human_closed['number'].astype(str) 
    + '/comments'
)

agentic_rejected_human_closed['pr_comment_url'] = (
    agentic_rejected_human_closed['repo_url'] 
    + '/issues/' 
    + agentic_rejected_human_closed['number'].astype(str) 
    + '/comments'
)

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================
GITHUB_TOKEN = "INSERT-TOKEN"

# --- NEW: CUTOFF DATE FOR REPRODUCIBILITY ---
# Only data created on or before this date will be included.
# Format: YYYY-MM-DD
CUTOFF_DATE = "2025-12-19" 

headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

if GITHUB_TOKEN == "INSERT-TOKEN":
    print("⚠️ WARNING: Token not configured.")
else:
    print(f"Configuration loaded. Cutoff date set to: {CUTOFF_DATE}")

Configuration loaded. Cutoff date set to: 2025-12-19


In [6]:
# =============================================================================
# FETCH FUNCTIONS
# =============================================================================

def fetch_data_robust(url):
    if not url or pd.isna(url):
        return []
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 403 and 'X-RateLimit-Reset' in response.headers:
            reset_time = int(response.headers['X-RateLimit-Reset'])
            sleep_time = reset_time - time.time() + 10
            if sleep_time > 0:
                print(f"\n⛔ Rate Limit! Sleeping {sleep_time/60:.1f} min...")
                time.sleep(sleep_time)
                return fetch_data_robust(url)
        
        if response.status_code == 200:
            return response.json()
        return []
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return []

def extract_review_comments(url):
    data = fetch_data_robust(url)
    extracted = []
    
    for c in data:
        created_at = c.get('created_at', '')
        if created_at[:10] > CUTOFF_DATE:
            continue 
            
        user_data = c.get('user') or {}
        extracted.append({
            'id': c.get('id'), # <--- ID ÚNICO DO COMENTÁRIO
            'user_login': user_data.get('login'),
            'user_type': user_data.get('type'),
            'body': c.get('body'),
            'author_association': c.get('author_association'),
            'created_at': created_at,
            'url': c.get('html_url')
        })
    return extracted

def extract_issue_comments(url):
    data = fetch_data_robust(url)
    extracted = []
    
    for c in data:
        created_at = c.get('created_at', '')
        if created_at[:10] > CUTOFF_DATE:
            continue

        user_data = c.get('user') or {}
        app_data = c.get('performed_via_github_app')
        
        extracted.append({
            'id': c.get('id'), # <--- ID ÚNICO DO COMENTÁRIO
            'user_login': user_data.get('login'),
            'user_type': user_data.get('type'),
            'body': c.get('body'),
            'author_association': c.get('author_association'),
            'app_slug': app_data.get('slug') if app_data else None,
            'app_name': app_data.get('name') if app_data else None,
            'created_at': created_at,
            'url': c.get('html_url')
        })
    return extracted

In [7]:
print(f"1. Fetching Code Review Comments (<= {CUTOFF_DATE})...")
review_comments_list = [
    extract_review_comments(url) 
    for url in tqdm(agentic_rejected_human_closed['review_comment_url'])
]
agentic_rejected_human_closed['review_comments_data'] = review_comments_list

print(f"\n2. Fetching General Issue Comments (<= {CUTOFF_DATE})...")
pr_comments_list = [
    extract_issue_comments(url) 
    for url in tqdm(agentic_rejected_human_closed['pr_comment_url'])
]
agentic_rejected_human_closed['pr_comments_data'] = pr_comments_list

print("\nDone!")

1. Fetching Code Review Comments (<= 2025-12-19)...


100%|█████████████████████████████████████████| 365/365 [03:51<00:00,  1.58it/s]



2. Fetching General Issue Comments (<= 2025-12-19)...


100%|█████████████████████████████████████████| 365/365 [03:30<00:00,  1.73it/s]


Done!


In [8]:
# Create a subset dataframe for processing
comments_dataset = agentic_rejected_human_closed[[
    'id', 'number', 'agent', 'html_url',
    'reviews_data',         # From Step 4
    'review_comments_data', # Fetched just now
    'pr_comments_data'      # Fetched just now
]].copy()

In [9]:

def parse_and_filter_reviews(x):
    if pd.isna(x) or x == "":
        return []
    try:
        raw_list = ast.literal_eval(x)
        filtered = []
        for r in raw_list:
            submitted_at = r.get('submitted_at', '')
            if submitted_at and submitted_at[:10] <= CUTOFF_DATE:
                filtered.append(r)
        return filtered
    except:
        return []

comments_dataset["reviews_data_parsed"] = comments_dataset["reviews_data"].apply(parse_and_filter_reviews)

exploded_reviews = (
    comments_dataset
    .explode("reviews_data_parsed")
    .reset_index(drop=True)
)

reviews_normalized = pd.json_normalize(exploded_reviews["reviews_data_parsed"])

final_all_reviews = pd.concat(
    [exploded_reviews.drop(columns=["reviews_data", "reviews_data_parsed", "review_comments_data", "pr_comments_data"]), 
     reviews_normalized], 
    axis=1
)

if 'body' in final_all_reviews.columns:
    final_all_reviews = final_all_reviews[final_all_reviews['body'].notna() & (final_all_reviews['body'] != '')]
    final_all_reviews = final_all_reviews[~final_all_reviews['user'].str.contains('bot', case=False, na=False)]

print(f"Extracted {len(final_all_reviews)} review verdicts.")

Extracted 139 review verdicts.


In [10]:
exploded_rc = (
    comments_dataset[comments_dataset["review_comments_data"].str.len() > 2]
    .explode("review_comments_data")
    .reset_index(drop=True)
)

rc_normalized = pd.json_normalize(exploded_rc["review_comments_data"])

final_review_comments = pd.concat(
    [exploded_rc.drop(columns=["reviews_data", "review_comments_data", "pr_comments_data"]),
     rc_normalized],
    axis=1
)

if 'id' in final_review_comments.columns:
    before = len(final_review_comments)
    final_review_comments = final_review_comments.drop_duplicates(subset=['id'])
    print(f"Code Comments: Removed {before - len(final_review_comments)} duplicates.")

print(f"Final Code Review Comments: {len(final_review_comments)}")

Code Comments: Removed 0 duplicates.
Final Code Review Comments: 590


In [11]:
exploded_prc = (
    comments_dataset[comments_dataset["pr_comments_data"].str.len() > 2]
    .explode("pr_comments_data")
    .reset_index(drop=True)
)

prc_normalized = pd.json_normalize(exploded_prc["pr_comments_data"])

final_pr_comments = pd.concat(
    [exploded_prc.drop(columns=["reviews_data", "review_comments_data", "pr_comments_data"]),
     prc_normalized],
    axis=1
)

cols_drop = ['app_slug', 'app_name']
final_pr_comments.drop(columns=[c for c in cols_drop if c in final_pr_comments.columns], inplace=True)

if 'id' in final_pr_comments.columns:
    before = len(final_pr_comments)
    final_pr_comments = final_pr_comments.drop_duplicates(subset=['id'])
    print(f"General Comments: Removed {before - len(final_pr_comments)} duplicates.")

print(f"Final General Comments: {len(final_pr_comments)}")

General Comments: Removed 0 duplicates.
Final General Comments: 795


In [12]:
# =============================================================================
# SAVE RESULTS
# =============================================================================

# Ensure directory exists
output_dir = 'output_files/comments'
os.makedirs(output_dir, exist_ok=True)

# 1. Save Review Verdicts (Approved/Changes Requested)
final_all_reviews.to_csv(f'{output_dir}/comments_extracted_from_review_type.csv', index=False)
print(f"✅ Saved: {output_dir}/comments_extracted_from_review_type.csv")

# 2. Save Code Comments (Inline) - Only Humans
if 'user_type' in final_review_comments.columns:
    human_rc = final_review_comments[final_review_comments['user_type'] == 'User']
    human_rc.to_csv(f'{output_dir}/comments_extracted_from_review_comment.csv', index=False)
    print(f"✅ Saved: {output_dir}/comments_extracted_from_review_comment.csv")

# 3. Save General Comments - Only Humans
if 'user_type' in final_pr_comments.columns:
    human_prc = final_pr_comments[final_pr_comments['user_type'] == 'User']
    human_prc.to_csv(f'{output_dir}/comments_extracted_from_pr_comment.csv', index=False)
    print(f"✅ Saved: {output_dir}/comments_extracted_from_pr_comment.csv")

✅ Saved: output_files/comments/comments_extracted_from_review_type.csv
✅ Saved: output_files/comments/comments_extracted_from_review_comment.csv
✅ Saved: output_files/comments/comments_extracted_from_pr_comment.csv
